<a href="https://colab.research.google.com/github/Rajesh24mcs115/24mcs115-Exp3-RajeshKumarPal/blob/main/Experiment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 24mcs115 (24mcs115-national-institute-of-technology-hamirpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 24mcs115 (24mcs115-national-institute-of-technology-hamirpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


acc,▃▁▄▅█▆▇▇
loss,█▃▂▂▂▁▂▁
acc,0.72423
loss,0.24891


In [ ]:
import h5py
with h5py.File('/content/sample_data/usps.h5', 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]


In [ ]:
# Use split:  1,000 of them as training data, another 1,000 as validation data, and the remaining 9,000 as test data
# Output: Which of the 10 different digits it is,
# Use a 10-way softmax as the output layer of the  Neural Network.
# The input layer : 256 units, i.e. one for each pixel.
# One  hidden layer of logistic units
# Investigate  what number of hidden units works best for generalization.
# Compute  i) cross-entropy error  ii) classification error rate.
# For  gradient descent with momentum optimizer, find good values for the learning rate and the momentum multiplier.

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

# Assuming X_tr and y_tr are already defined from the previous code.

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(X_tr, y_tr, train_size=1000, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=9000/10000, random_state=42) # 9000 test samples


# Define the neural network architecture and training loop
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def sigmoid(self, z):
      return 1 / (1 + np.exp(-z))

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True)) # Numerical stability
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def forward(self, X):
        z1 = X @ self.W1 + self.b1
        a1 = self.sigmoid(z1)
        z2 = a1 @ self.W2 + self.b2
        a2 = self.softmax(z2)
        return a1, a2

    def predict(self,X):
        _, output = self.forward(X)
        return np.argmax(output, axis=1)

    def train(self, X_train, y_train, X_val, y_val, learning_rate, momentum, epochs):

        # Initialize momentum terms
        v_W1 = np.zeros_like(self.W1)
        v_b1 = np.zeros_like(self.b1)
        v_W2 = np.zeros_like(self.W2)
        v_b2 = np.zeros_like(self.b2)

        for epoch in range(epochs):
            # Forward pass
            a1, output = self.forward(X_train)

            # Compute loss and accuracy
            loss = log_loss(y_train, output) # cross entropy
            predictions = self.predict(X_train)
            accuracy = accuracy_score(y_train, predictions)


            # Backpropagation (calculate gradients here)  ... (Implementation omitted for brevity)


            # Gradient descent with momentum

            # ... (Update weights and biases using gradients and momentum terms, implementation omitted)

            # Print training progress
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")


input_size = 256
hidden_size = 64 # Example hidden units, need to experiment with multiple values
output_size = 10
learning_rate = 0.01  # Experiment to find good learning rate
momentum = 0.9     # Experiment to find good momentum


nn = NeuralNetwork(input_size, hidden_size, output_size)
nn.train(X_train, y_train, X_val, y_val, learning_rate, momentum, epochs=10)


# Evaluate on test data
test_predictions = nn.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)
test_loss = log_loss(y_test, nn.forward(X_test)[1])
print(f"\nTest Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")



Epoch 1/10, Loss: 2.3125, Accuracy: 0.0740
Epoch 2/10, Loss: 2.3125, Accuracy: 0.0740
Epoch 3/10, Loss: 2.3125, Accuracy: 0.0740
Epoch 4/10, Loss: 2.3125, Accuracy: 0.0740
Epoch 5/10, Loss: 2.3125, Accuracy: 0.0740
Epoch 6/10, Loss: 2.3125, Accuracy: 0.0740
Epoch 7/10, Loss: 2.3125, Accuracy: 0.0740
Epoch 8/10, Loss: 2.3125, Accuracy: 0.0740
Epoch 9/10, Loss: 2.3125, Accuracy: 0.0740
Epoch 10/10, Loss: 2.3125, Accuracy: 0.0740

Test Accuracy: 0.0940, Test Loss: 2.3084


In [7]:
# 1) Calculate classification loss on the validation data with different combinations  (eg, Initializing with zero weight decay, 200 hidden units, 1000 optimization iterations, a learning rate of 0.40, momentum of 0.9, no early stopping, and mini-batch size 100)

import numpy as np
from sklearn.metrics import log_loss

def train_and_evaluate(hidden_size, learning_rate, momentum, epochs, weight_decay=0, early_stopping=False, mini_batch_size=100):
    nn = NeuralNetwork(input_size, hidden_size, output_size)
    nn.train(X_train, y_train, X_val, y_val, learning_rate, momentum, epochs)

    _, val_predictions = nn.forward(X_val)
    val_loss = log_loss(y_val, val_predictions)

    return val_loss


# Example usage with different combinations of hyperparameters
hyperparameter_combinations = [
    {"hidden_size": 200, "learning_rate": 0.40, "momentum": 0.9, "epochs": 1000, "weight_decay": 0},
    # Add more combinations here
    {"hidden_size": 100, "learning_rate": 0.1, "momentum": 0.95, "epochs": 500, "weight_decay": 0.001},
]

# Initialize wandb before logging hyperparameter combinations
wandb.init(project="Ex-3", config={"architecture": "NeuralNetwork", "dataset": "USPS"}) #Reinitialize wandb with a new run

for combination in hyperparameter_combinations:
    val_loss = train_and_evaluate(**combination)
    print(f"Hyperparameters: {combination}, Validation Loss: {val_loss}")

    # Log to Weights and Biases
    wandb.log({"val_loss": val_loss, **combination})

wandb.finish()

Epoch 1/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 2/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 3/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 4/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 5/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 6/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 7/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 8/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 9/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 10/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 11/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 12/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 13/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 14/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 15/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 16/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 17/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 18/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 19/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 20/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 21/1000, Loss: 2.3121, Accuracy: 0.0890
Epoch 22/1000, Loss: 2.3121, Accuracy: 0.08

epochs,█▁
hidden_size,█▁
learning_rate,█▁
momentum,▁█
val_loss,█▁
weight_decay,▁█
epochs,500
hidden_size,100
learning_rate,0.1
momentum,0.95
val_loss,2.30568


In [8]:
# 2) Investigate weight decay using  L2  weight decay(WD) with different coefficients .
# Run experiments with the other WD coefficients listed below (not limited), and indicate which of them gave the best generalization. Focus on the classification loss (i.e. without the weight decay loss), as opposed to the final loss (which does include the weight decay loss).
# 0.0
# 0.0001
# 0.001
# 0.01
# 1
# 2
# 5

import wandb
import random
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

def train_and_evaluate(hidden_size, learning_rate, momentum, epochs, weight_decay=0, early_stopping=False, mini_batch_size=100):
    nn = NeuralNetwork(input_size, hidden_size, output_size)

    # Initialize momentum terms
    v_W1 = np.zeros_like(nn.W1)
    v_b1 = np.zeros_like(nn.b1)
    v_W2 = np.zeros_like(nn.W2)
    v_b2 = np.zeros_like(nn.b2)

    for epoch in range(epochs):
        # Forward pass
        a1, output = nn.forward(X_train)

        # Compute loss and accuracy
        loss = log_loss(y_train, output) # cross entropy
        predictions = nn.predict(X_train)
        accuracy = accuracy_score(y_train, predictions)

        #L2 regularization term
        l2_reg = (np.sum(nn.W1**2) + np.sum(nn.W2**2)) * weight_decay / 2

        #Backpropagation (calculate gradients here)
        delta2 = output - np.eye(10)[y_train] # Assuming 10 output classes
        dW2 = a1.T @ delta2 + weight_decay * nn.W2 #Added weight decay to the gradient
        db2 = np.sum(delta2, axis=0, keepdims=True)
        delta1 = delta2 @ nn.W2.T * a1 * (1 - a1)
        dW1 = X_train.T @ delta1+ weight_decay * nn.W1 #Added weight decay to the gradient
        db1 = np.sum(delta1, axis=0, keepdims=True)


        # Gradient descent with momentum
        v_W1 = momentum * v_W1 - learning_rate * dW1
        v_b1 = momentum * v_b1 - learning_rate * db1
        v_W2 = momentum * v_W2 - learning_rate * dW2
        v_b2 = momentum * v_b2 - learning_rate * db2

        nn.W1 += v_W1
        nn.b1 += v_b1
        nn.W2 += v_W2
        nn.b2 += v_b2

        # Print training progress
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, L2 reg: {l2_reg:.4f}")


    _, val_predictions = nn.forward(X_val)
    val_loss = log_loss(y_val, val_predictions)

    return val_loss

# Example usage with different weight decay coefficients
weight_decay_values = [0.0, 0.0001, 0.001, 0.01, 1, 2, 5]

# Initialize wandb before logging hyperparameter combinations
wandb.init(project="Ex-3-WeightDecay", config={"architecture": "NeuralNetwork", "dataset": "USPS"})

for wd in weight_decay_values:
    val_loss = train_and_evaluate(hidden_size=100, learning_rate=0.1, momentum=0.95, epochs=10, weight_decay=wd)
    print(f"Weight Decay: {wd}, Validation Loss: {val_loss}")

    # Log to Weights and Biases
    wandb.log({"val_loss": val_loss, "weight_decay": wd})

wandb.finish()


Epoch 1/10, Loss: 2.3088, Accuracy: 0.0890, L2 reg: 0.0000
Epoch 2/10, Loss: 29.7721, Accuracy: 0.1740, L2 reg: 0.0000
Epoch 3/10, Loss: 15.4070, Accuracy: 0.1300, L2 reg: 0.0000
Epoch 4/10, Loss: 17.9711, Accuracy: 0.1050, L2 reg: 0.0000
Epoch 5/10, Loss: 17.6910, Accuracy: 0.0890, L2 reg: 0.0000
Epoch 6/10, Loss: 20.6101, Accuracy: 0.0840, L2 reg: 0.0000


<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3a

Epoch 7/10, Loss: 25.8406, Accuracy: 0.0760, L2 reg: 0.0000
Epoch 8/10, Loss: 28.9527, Accuracy: 0.0740, L2 reg: 0.0000
Epoch 9/10, Loss: 30.4980, Accuracy: 0.1030, L2 reg: 0.0000
Epoch 10/10, Loss: 33.0160, Accuracy: 0.0840, L2 reg: 0.0000
Weight Decay: 0.0, Validation Loss: 30.400181158019084


<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Epoch 1/10, Loss: 2.3026, Accuracy: 0.0890, L2 reg: 0.0001
Epoch 2/10, Loss: 29.7721, Accuracy: 0.1740, L2 reg: 0.1088
Epoch 3/10, Loss: 15.0395, Accuracy: 0.1300, L2 reg: 599.7731
Epoch 4/10, Loss: 17.8221, Accuracy: 0.1050, L2 reg: 2282.8956


...

Epoch 7/10, Loss: 26.4985, Accuracy: 0.0840, L2 reg: 54364477.5776
Epoch 8/10, Loss: 29.4320, Accuracy: 0.0810, L2 reg: 45527623.1824
Epoch 9/10, Loss: 30.7541, Accuracy: 0.0760, L2 reg: 30010337.4728
Epoch 10/10, Loss: 33.3764, Accuracy: 0.0740, L2 reg: 13872607.4622
Weight Decay: 1, Validation Loss: 31.755430878949017
Epoch 1/10, Loss: 2.3015, Accuracy: 0.0840, L2 reg: 2.6838
Epoch 2/10, Loss: 29.7721, Accuracy: 0.1740, L2 reg: 2104.8610
Epoch 3/10, Loss: 14.7123, Accuracy: 0.1300, L2 reg: 11714414.2686
Epoch 4/10, Loss: 17.8271, Accuracy: 0.1050, L2 reg: 35918588.8650
Epoch 5/10, Loss: 19.2382, Accuracy: 0.1030, L2 reg: 52370458.4973


<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3a

Epoch 6/10, Loss: 21.7848, Accuracy: 0.0890, L2 reg: 48592121.4806
Epoch 7/10, Loss: 26.8372, Accuracy: 0.0840, L2 reg: 28345592.8592
Epoch 8/10, Loss: 29.1254, Accuracy: 0.0810, L2 reg: 7262872.9727
Epoch 9/10, Loss: 31.0369, Accuracy: 0.0760, L2 reg: 118427.6400
Epoch 10/10, Loss: 29.7721, Accuracy: 0.1740, L2 reg: 9977302.8701
Weight Decay: 2, Validation Loss: 33.063891900668665
Epoch 1/10, Loss: 2.2944, Accuracy: 0.1300, L2 reg: 6.7504
Epoch 2/10, Loss: 29.7721, Accuracy: 0.1740, L2 reg: 4443.3259
Epoch 3/10, Loss: 12.8538, Accuracy: 0.1300, L2 reg: 27539688.0781
Epoch 4/10, Loss: 17.3051, Accuracy: 0.1050, L2 reg: 57998584.4266
Epoch 5/10, Loss: 17.5015, Accuracy: 0.0840, L2 reg: 36694662.1637
Epoch 6/10, Loss: 22.1573, Accuracy: 0.0810, L2 reg: 2401238.0856
Epoch 7/10, Loss: 29.7380, Accuracy: 0.1740, L2 reg: 12317894.9799
Epoch 8/10, Loss: 33.0851, Accuracy: 0.0740, L2 reg: 42685772.8135
Epoch 9/10, Loss: 31.3580, Accuracy: 0.1300, L2 reg: 40730430.9279
Epoch 10/10, Loss: 32.331

<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


val_loss,▂▄█▂▅▇▁
weight_decay,▁▁▁▁▂▄█
val_loss,29.83329
weight_decay,5


In [9]:
# 3) Experiment with the number of model parameters. Vary the number of hidden units.
# Turn off the weight decay, and try the following hidden layer sizes. Indicate which one worked best.
# 10
# 30
# 100
# 130
# 170
# 200

import wandb
import random
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

# Assuming previous code and imports are present

hidden_layer_sizes = [10, 30, 100, 130, 170, 200]
best_hidden_size = None
best_val_loss = float('inf')

wandb.init(project="Ex-3-HiddenUnits", config={"architecture": "NeuralNetwork", "dataset": "USPS"})

for hidden_size in hidden_layer_sizes:
    val_loss = train_and_evaluate(hidden_size=hidden_size, learning_rate=0.1, momentum=0.95, epochs=10, weight_decay=0)
    print(f"Hidden Size: {hidden_size}, Validation Loss: {val_loss}")

    wandb.log({"val_loss": val_loss, "hidden_size": hidden_size})

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_hidden_size = hidden_size

print(f"\nBest hidden layer size: {best_hidden_size} with validation loss: {best_val_loss}")

wandb.finish()


Epoch 1/10, Loss: 2.3024, Accuracy: 0.0880, L2 reg: 0.0000
Epoch 2/10, Loss: 22.9230, Accuracy: 0.1740, L2 reg: 0.0000
Epoch 3/10, Loss: 15.0872, Accuracy: 0.1300, L2 reg: 0.0000
Epoch 4/10, Loss: 17.7240, Accuracy: 0.1050, L2 reg: 0.0000
Epoch 5/10, Loss: 20.4747, Accuracy: 0.0890, L2 reg: 0.0000
Epoch 6/10, Loss: 23.7371, Accuracy: 0.0840, L2 reg: 0.0000
Epoch 7/10, Loss: 28.8973, Accuracy: 0.0810, L2 reg: 0.0000
Epoch 8/10, Loss: 30.9427, Accuracy: 0.0760, L2 reg: 0.0000
Epoch 9/10, Loss: 33.3764, Accuracy: 0.0740, L2 reg: 0.0000
Epoch 10/10, Loss: 30.4038, Accuracy: 0.1030, L2 reg: 0.0000
Hidden Size: 10, Validation Loss: 27.27403418980618
Epoch 1/10, Loss: 2.3022, Accuracy: 0.0740, L2 reg: 0.0000
Epoch 2/10, Loss: 29.7721, Accuracy: 0.1740, L2 reg: 0.0000
Epoch 3/10, Loss: 14.6294, Accuracy: 0.1300, L2 reg: 0.0000
Epoch 4/10, Loss: 17.8268, Accuracy: 0.1050, L2 reg: 0.0000
Epoch 5/10, Loss: 18.4625, Accuracy: 0.0890, L2 reg: 0.0000
Epoch 6/10, Loss: 23.8731, Accuracy: 0.0840, L2 r

<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3a

Epoch 8/10, Loss: 31.1473, Accuracy: 0.0760, L2 reg: 0.0000
Epoch 9/10, Loss: 33.3764, Accuracy: 0.0740, L2 reg: 0.0000
Epoch 10/10, Loss: 28.0514, Accuracy: 0.0740, L2 reg: 0.0000
Hidden Size: 30, Validation Loss: 30.311665599391038
Epoch 1/10, Loss: 2.3066, Accuracy: 0.1030, L2 reg: 0.0000
Epoch 2/10, Loss: 29.7721, Accuracy: 0.1740, L2 reg: 0.0000
Epoch 3/10, Loss: 15.8252, Accuracy: 0.1300, L2 reg: 0.0000
Epoch 4/10, Loss: 19.4674, Accuracy: 0.1050, L2 reg: 0.0000
Epoch 5/10, Loss: 22.9818, Accuracy: 0.1030, L2 reg: 0.0000
Epoch 6/10, Loss: 23.3743, Accuracy: 0.0890, L2 reg: 0.0000
Epoch 7/10, Loss: 25.3909, Accuracy: 0.0840, L2 reg: 0.0000


<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3a

Epoch 8/10, Loss: 27.5962, Accuracy: 0.0840, L2 reg: 0.0000
Epoch 9/10, Loss: 28.3905, Accuracy: 0.0760, L2 reg: 0.0000
Epoch 10/10, Loss: 33.3764, Accuracy: 0.0740, L2 reg: 0.0000
Hidden Size: 100, Validation Loss: 28.339033736839493
Epoch 1/10, Loss: 2.3093, Accuracy: 0.0810, L2 reg: 0.0000
Epoch 2/10, Loss: 29.7721, Accuracy: 0.1740, L2 reg: 0.0000
Epoch 3/10, Loss: 15.0979, Accuracy: 0.1300, L2 reg: 0.0000
Epoch 4/10, Loss: 17.7141, Accuracy: 0.1030, L2 reg: 0.0000


...

Epoch 4/10, Loss: 18.5452, Accuracy: 0.1050, L2 reg: 0.0000
Epoch 5/10, Loss: 20.8552, Accuracy: 0.0890, L2 reg: 0.0000
Epoch 6/10, Loss: 23.5991, Accuracy: 0.0840, L2 reg: 0.0000
Epoch 7/10, Loss: 26.7411, Accuracy: 0.0810, L2 reg: 0.0000
Epoch 8/10, Loss: 30.2292, Accuracy: 0.0760, L2 reg: 0.0000
Epoch 9/10, Loss: 32.5575, Accuracy: 0.0740, L2 reg: 0.0000


<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
<ipython-input-6-3a

Epoch 10/10, Loss: 31.1051, Accuracy: 0.1030, L2 reg: 0.0000
Hidden Size: 200, Validation Loss: 32.31895152855655

Best hidden layer size: 10 with validation loss: 27.27403418980618


<ipython-input-6-3ae2e72d1e47>:30: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


hidden_size,▁▂▄▅▇█
val_loss,▁▄▂█▆▇
hidden_size,200
val_loss,32.31895


In [10]:
# 4) Combine early stopping with a carefully chosen hidden layer size. Which number of hidden units works best that way, i.e. with early stopping?
import numpy as np
from sklearn.metrics import log_loss

def train_and_evaluate(hidden_size, learning_rate, momentum, epochs, weight_decay=0, early_stopping=True, patience=5, mini_batch_size=100):
    nn = NeuralNetwork(input_size, hidden_size, output_size)
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(epochs):

        _, val_predictions = nn.forward(X_val)
        val_loss = log_loss(y_val, val_predictions)

        print(f"Epoch {epoch + 1}/{epochs}, Val Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_weights = {  # Save best weights
                "W1": nn.W1.copy(),
                "b1": nn.b1.copy(),
                "W2": nn.W2.copy(),
                "b2": nn.b2.copy()
            }
        else:
            epochs_no_improve += 1

        if early_stopping and epochs_no_improve == patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    # Load best weights
    nn.W1 = best_weights["W1"]
    nn.b1 = best_weights["b1"]
    nn.W2 = best_weights["W2"]
    nn.b2 = best_weights["b2"]

    return best_val_loss


hidden_layer_sizes = [10, 30, 100, 130, 170, 200]
best_hidden_size = None
best_val_loss = float('inf')

wandb.init(project="Ex-3-HiddenUnits-EarlyStopping", config={"architecture": "NeuralNetwork", "dataset": "USPS"})

for hidden_size in hidden_layer_sizes:
    val_loss = train_and_evaluate(hidden_size=hidden_size, learning_rate=0.1, momentum=0.95, epochs=100, weight_decay=0, early_stopping=True) # Increased epochs
    print(f"Hidden Size: {hidden_size}, Validation Loss: {val_loss}")

    wandb.log({"val_loss": val_loss, "hidden_size": hidden_size})

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_hidden_size = hidden_size

print(f"\nBest hidden layer size with early stopping: {best_hidden_size} with validation loss: {best_val_loss}")

wandb.finish()

Epoch 1/100, Val Loss: 2.3050
Epoch 2/100, Val Loss: 2.3050
Epoch 3/100, Val Loss: 2.3050
Epoch 4/100, Val Loss: 2.3050
Epoch 5/100, Val Loss: 2.3050
Epoch 6/100, Val Loss: 2.3050
Early stopping triggered at epoch 6
Hidden Size: 10, Validation Loss: 2.304969531455792
Epoch 1/100, Val Loss: 2.3031
Epoch 2/100, Val Loss: 2.3031
Epoch 3/100, Val Loss: 2.3031
Epoch 4/100, Val Loss: 2.3031
Epoch 5/100, Val Loss: 2.3031
Epoch 6/100, Val Loss: 2.3031
Early stopping triggered at epoch 6
Hidden Size: 30, Validation Loss: 2.3031174229123836
Epoch 1/100, Val Loss: 2.2998
Epoch 2/100, Val Loss: 2.2998
Epoch 3/100, Val Loss: 2.2998
Epoch 4/100, Val Loss: 2.2998
Epoch 5/100, Val Loss: 2.2998
Epoch 6/100, Val Loss: 2.2998
Early stopping triggered at epoch 6
Hidden Size: 100, Validation Loss: 2.299788111273319
Epoch 1/100, Val Loss: 2.3015
Epoch 2/100, Val Loss: 2.3015
Epoch 3/100, Val Loss: 2.3015
Epoch 4/100, Val Loss: 2.3015
Epoch 5/100, Val Loss: 2.3015
Epoch 6/100, Val Loss: 2.3015
Early stopping

hidden_size,▁▂▄▅▇█
val_loss,▅▄▁▂▇█
hidden_size,200
val_loss,2.30861
